In [92]:
import requests
from bs4 import BeautifulSoup
import re
import sqlite3
import time

WIKI_URL = "https://en.wikipedia.org/wiki/List_of_highest-grossing_films"

# We'll look for these table captions.
MAIN_TABLE_CAPTIONS = [
    "Highest-grossing films",  # The primary table listing top films
    # Optionally include other tables from the page if needed:
    #"Highest-grossing films as of 2023 adjusted for inflation",
    #"High-grossing films by year of release",
    #"Timeline of the highest-grossing film record",
    #"Highest-grossing franchises and film series"
]

## Create (or connect to) SQLite database

In [93]:
# Create or connect to SQLite DB
conn = sqlite3.connect("films.db")
cursor = conn.cursor()

# Create the films table if not exists
cursor.execute("""
    CREATE TABLE IF NOT EXISTS films (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT NOT NULL,
        release_year INTEGER,
        director TEXT,
        box_office TEXT,  -- Storing as TEXT for less complexity (symbols, etc.)
        country TEXT
    )
""")
conn.commit()

##  Helper functions to parse each table

In [94]:
def get_infobox_value(infobox, label):
    """
    Searches the infobox for a <th> whose text contains `label` 
    (case-insensitive). Returns the text content of the adjacent <td> if found.
    """
    if not infobox or not label:
        return None
    row = infobox.find("th", string=lambda text: text and label.lower() in text.lower())
    if row:
        data_cell = row.find_next_sibling("td")
        if data_cell:
            # Combine all text in that cell
            return " ".join(data_cell.stripped_strings)
    return None

In [95]:
def parse_directors(director_str):
    """
    Given the 'Directed by' cell's string, parse out multiple directors if present,
    returning a comma-separated string.
    """
    if not director_str:
        return None
    # Split on typical separators: commas, 'and', '&'
    # This is simplistic—adjust as needed for edge cases.
    possible_separators = r",| and | & "
    directors_list = re.split(possible_separators, director_str)
    # Clean extra spaces
    directors_list = [d.strip() for d in directors_list if d.strip()]
    # Rejoin in a single comma-separated string
    return ", ".join(directors_list)

In [96]:
def parse_year_from_release_date(date_str):
    """
    Attempt to extract a 4-digit year from a 'Release date' string.
    Fallback to None if not found.
    """
    if not date_str:
        return None
    match = re.search(r"\b(19|20)\d{2}\b", date_str)
    if match:
        return int(match.group(0))
    return None

In [97]:
def parse_box_office(box_office_str):
    """
    Return the raw string or do some minimal cleanup. 
    We could remove symbols and parse as float, but some pages might have multiple
    numbers or ranges. For simplicity, store as TEXT or do partial numeric cleaning.
    """
    if not box_office_str:
        return None
    # Example: remove footnote references like [1]
    cleaned = re.sub(r"\[\d+\]", "", box_office_str)
    cleaned = cleaned.strip()
    return cleaned if cleaned else None

In [98]:
def parse_country(country_str):
    """
    Clean up the 'Country' field from the infobox if present.
    """
    if not country_str:
        return None
    # Remove possible footnote brackets, etc.
    cleaned = re.sub(r"\[\d+\]", "", country_str).strip()
    return cleaned

In [99]:
def scrape_film_page(film_url):
    """
    Given a partial or full Wikipedia link to a specific film page:
    1. Request the page
    2. Parse the infobox
    3. Extract director(s), release year, box office, and country
    4. Return these as a dict
    """
    # Some links might already contain the full domain, others might be relative
    if film_url.startswith("/wiki/"):
        film_url = "https://en.wikipedia.org" + film_url

    # Polite delay to avoid rate-limiting (optional but recommended)
    time.sleep(1)

    resp = requests.get(film_url)
    if resp.status_code != 200:
        print(f"Warning: Could not fetch {film_url}")
        return {}

    film_soup = BeautifulSoup(resp.text, "html.parser")

    # Locate the standard film infobox, typically class="infobox vevent"
    infobox = film_soup.find("table", class_="infobox vevent")
    if not infobox:
        # Some pages might not have a standard infobox
        return {}

    # Extract data from the infobox
    raw_director = get_infobox_value(infobox, "Directed by")
    director = parse_directors(raw_director)

    raw_release_date = get_infobox_value(infobox, "Release date") or \
                       get_infobox_value(infobox, "Release dates") or \
                       get_infobox_value(infobox, "Released")

    release_year = parse_year_from_release_date(raw_release_date)

    raw_box_office = get_infobox_value(infobox, "Box office")
    box_office = parse_box_office(raw_box_office)

    raw_country = get_infobox_value(infobox, "Country")
    country = parse_country(raw_country)

    return {
        "director": director,
        "release_year": release_year,
        "box_office": box_office,
        "country": country
    }

## SCRAPE THE MAIN PAGE & PARSE TARGET TABLE(S)

In [100]:
response = requests.get(WIKI_URL)
soup = BeautifulSoup(response.text, "html.parser")

# Identify the relevant tables by caption
tables = []
for caption_tag in soup.find_all('caption'):
    caption_text = caption_tag.get_text(strip=True)
    if any(main_caption in caption_text for main_caption in MAIN_TABLE_CAPTIONS):
        candidate_table = caption_tag.find_parent('table')
        tables.append(candidate_table)

print(f"Found {len(tables)} relevant table(s) to parse.")

Found 2 relevant table(s) to parse.


In [101]:
# For each relevant table, we parse rows
for tbl in tables:
    rows = tbl.find_all("tr")
    # Usually skip the header row
    for row in rows[1:]:
        cells = row.find_all(["th","td"])
        if not cells:
            continue
        
        # Attempt to locate the film link in the 'Title' cell
        # Some tables have the film title in a <th scope="row">,
        # others might be in a <td>. Adjust as needed.

        film_link_tag = None
        
        # A common pattern: the 2nd or 3rd cell might have the film <a> tag
        # Or we check for <th scope="row"> with <a> inside
        # We'll just search for any <a> inside the row that leads to a wiki film page:
        for cell in cells:
            link = cell.find("a", href=True)
            if link and link['href'].startswith("/wiki/"):
                film_link_tag = link
                break
        
        if not film_link_tag:
            continue
        
        film_title = film_link_tag.get_text(strip=True)
        film_href = film_link_tag['href']

        # Now open that film page to parse the infobox
        film_details = scrape_film_page(film_href)
        if not film_details:
            print(f"Could not parse infobox for film: {film_title}")
            continue
        
        director = film_details.get("director")
        release_year = film_details.get("release_year")
        box_office = film_details.get("box_office")
        country = film_details.get("country")

        # --------------------------------------------
        # 4. INSERT INTO DATABASE
        # --------------------------------------------
        cursor.execute("""
            INSERT INTO films (title, release_year, director, box_office, country)
            VALUES (?, ?, ?, ?, ?)
        """, (film_title, release_year, director, box_office, country))
        conn.commit()

        print(f"Inserted: {film_title} (Year={release_year}, Dir={director}, BoxOffice={box_office}, Country={country})")

# Cleanup
conn.close()
print("Done. Database populated with film data.")

Inserted: Avatar (Year=2009, Dir=James Cameron, BoxOffice=$2.923 billion [ 5 ], Country=None)
Inserted: Avengers: Endgame (Year=2019, Dir=Anthony Russo Joe Russo, BoxOffice=$2.799 billion [ 4 ], Country=United States)
Inserted: Avatar: The Way of Water (Year=2022, Dir=James Cameron, BoxOffice=$2.320 billion [ 4 ] [ 5 ], Country=United States)
Inserted: Titanic (Year=1997, Dir=James Cameron, BoxOffice=$2.264 billion [ 7 ], Country=United States)
Inserted: Star Wars: The Force Awakens (Year=2015, Dir=J. J. Abrams, BoxOffice=$2.07 billion [ 3 ], Country=United States)
Inserted: Avengers: Infinity War (Year=2018, Dir=Anthony Russo Joe Russo, BoxOffice=$2.052 billion [ 4 ], Country=United States)
Inserted: Ne Zha 2 (Year=2025, Dir=Jiaozi, BoxOffice=US$1.94 billion [ 2 ] [ 3 ], Country=China)
Inserted: Spider-Man: No Way Home (Year=2021, Dir=Jon Watts, BoxOffice=$1.923 billion [ 3 ] [ 4 ], Country=United States)
Inserted: Inside Out 2 (Year=2024, Dir=Kelsey Mann, BoxOffice=$1.699 billion [ 3

## Create the “Insights” Table

In [111]:
import sqlite3

conn = sqlite3.connect("films.db")
cursor = conn.cursor()

# Optional: create a table to hold per-film insights
cursor.execute("""
    CREATE TABLE IF NOT EXISTS film_insights (
        insight_id INTEGER PRIMARY KEY AUTOINCREMENT,
        film_id INTEGER,
        insight TEXT,
        FOREIGN KEY (film_id) REFERENCES films(id)
    )
""")

conn.commit()
conn.close()

## LLM Tools :)

In [112]:
def build_cleaning_prompt(film_dict):
    """
    Creates a text prompt that asks the LLM to clean the film data.
    Return a string that you'll send to the LLM endpoint.
    """
    prompt = f"""
    You are a data-cleaning assistant. I have the following film entry:
    
    Title: {film_dict["title"]}
    Release Year: {film_dict["release_year"]}
    Director(s): {film_dict["director"]}
    Box Office: {film_dict["box_office"]}
    Country: {film_dict["country"]}
    
    1. Remove bracketed references like [1], [ 2 ], or [ 3 ] from all fields.
    2. Convert Box Office to a consistent numeric format. 
       - For example, if the value is "$2.923 billion [ 5 ]", remove bracket references 
         and keep convert to numeric (e.g. 2923000000). 
    3. Standardize the Country field to something like "United States" or "United Kingdom" if it's partially included.
    4. Output your final result in valid JSON with the following keys exactly:
       title, release_year, director, box_office, country
       
    ONLY return the JSON object. Do not include other text.
    """
    return prompt


In [113]:
import aiohttp
import os

LLM_ENDPOINT = os.environ['LLM_ENDPOINT']

async def call_llm(prompt, 
             model="Meta-Llama-3.1-70B-Instruct", 
             temperature=0.7, 
             max_tokens=400, 
             top_p=0.9, 
             n=1):
    """
    Makes a POST request to your local LLM endpoint with the specified
    prompt and parameters. Returns the assistant's message content.
    """
    payload = {
        "messages": [
            {
                "role": "user",
                "content": prompt,
                "name": "user"
            }
        ],
        "model": model,
        "temperature": temperature,
        "max_tokens": max_tokens,
        "top_p": top_p,
        "n": n
    }

    async with aiohttp.ClientSession() as session:
        async with session.post(LLM_ENDPOINT, json=payload) as response:
            if response.status != 200:
                raise Exception(f"Error {response.status} from endpoint: {LLM_ENDPOINT}")
            
            result_json = await response.json()
            try:
                result = result_json["choices"][0]["message"]["content"]
            except Exception:
                result = str(result_json)
    
    return result

In [114]:
import sqlite3
import json
import asyncio

async def clean_and_update_films():
    conn = sqlite3.connect("films.db")
    cursor = conn.cursor()

    # Step 1: Get all records
    cursor.execute("SELECT id, title, release_year, director, box_office, country FROM films")
    rows = cursor.fetchall()

    # We'll store them in a list of dicts for convenience
    film_records = []
    for row in rows:
        film_records.append({
            "id": row[0],
            "title": row[1],
            "release_year": row[2],
            "director": row[3],
            "box_office": row[4],
            "country": row[5]
        })

    # Step 2: For each film, build a prompt and call LLM
    for film_dict in film_records:
        prompt = build_cleaning_prompt(film_dict)

        # Call your local LLM endpoint
        cleaned_response = await call_llm(
            prompt,
            model="Meta-Llama-3.1-70B-Instruct",
            temperature=0.7,
            max_tokens=400,
            top_p=0.9,
            n=1
        )

        # Step 3: Parse the JSON response
        try:
            cleaned_data = json.loads(cleaned_response)
            # cleaned_data should contain keys: title, release_year, director, box_office, country
        except json.JSONDecodeError:
            print(f"Error: Could not parse JSON for film ID {film_dict['id']}")
            continue

        # Let's handle cases where the LLM might not supply a field
        title = cleaned_data.get("title", film_dict["title"])
        release_year = cleaned_data.get("release_year", film_dict["release_year"])
        director = cleaned_data.get("director", film_dict["director"])
        box_office = cleaned_data.get("box_office", film_dict["box_office"])
        country = cleaned_data.get("country", film_dict["country"])

        try:
            cursor.execute("""
                UPDATE films
                SET title = ?, 
                    release_year = ?, 
                    director = ?, 
                    box_office = ?, 
                    country = ?
                WHERE id = ?
            """, (title, release_year, director, box_office, country, film_dict["id"]))
            conn.commit()
        except Exception as e:
            print(f"Error updating film ID {film_dict['id']}: {e}")
        
        insight_prompt = f"""
Give me a single short sentence insight about the film:
Title: {title}
Release Year: {release_year}
Director: {director}
Box Office: {box_office}
Country: {country}

Only return a single short sentence as plain text.
        """
        insight_text = await call_llm(
            insight_prompt,
            model="Meta-Llama-3.1-70B-Instruct",
            temperature=0.7,
            max_tokens=60,
            top_p=0.9,
            n=1
        )

        # Insert insight into film_insights table
        try:
            cursor.execute("""
                INSERT INTO film_insights (film_id, insight)
                VALUES (?, ?)
            """, (film_dict["id"], insight_text.strip()))
            conn.commit()
        except Exception as e:
            print(f"Error inserting insight for film ID {film_dict['id']}: {e}")

        print(f"Cleaned & updated film ID {film_dict['id']}: {title}")
        
    conn.close()
    print("All films processed and cleaned.")


# Finally, run the coroutine in an event loop (if you're in a normal .py script)
if __name__ == "__main__":
    asyncio.run(clean_and_update_films())

Cleaned & updated film ID 1: Avatar
Cleaned & updated film ID 2: Avengers: Endgame
Cleaned & updated film ID 3: Avatar: The Way of Water
Cleaned & updated film ID 4: Titanic
Cleaned & updated film ID 5: Star Wars: The Force Awakens
Cleaned & updated film ID 6: Avengers: Infinity War
Cleaned & updated film ID 7: Ne Zha 2
Cleaned & updated film ID 8: Spider-Man: No Way Home
Cleaned & updated film ID 9: Inside Out 2
Cleaned & updated film ID 10: Jurassic World
Cleaned & updated film ID 11: The Lion King
Cleaned & updated film ID 12: The Avengers
Cleaned & updated film ID 13: Furious 7
Cleaned & updated film ID 14: Top Gun: Maverick
Cleaned & updated film ID 15: Frozen 2
Cleaned & updated film ID 16: Barbie
Cleaned & updated film ID 17: Avengers: Age of Ultron
Cleaned & updated film ID 18: The Super Mario Bros. Movie
Cleaned & updated film ID 19: Black Panther
Cleaned & updated film ID 20: Harry Potter and the Deathly Hallows – Part 2
Cleaned & updated film ID 21: Deadpool & Wolverine
Clea

## Removing Duplicate Rows in films by title

In [115]:
import sqlite3

conn = sqlite3.connect("films.db")
cursor = conn.cursor()

# Step 1: Delete duplicates in 'films', keeping only the row with the MIN(id) for each title
cursor.execute("""
    DELETE FROM films
    WHERE id NOT IN (
        SELECT MIN(id)
        FROM films
        GROUP BY title
    )
""")

conn.commit()
conn.close()


##  Keeping Only One Entry per film_id in film_insights 

In [116]:
import sqlite3

conn = sqlite3.connect("films.db")
cursor = conn.cursor()

# Step 2: Delete duplicates in 'film_insights', keeping only the row with the MIN(insight_id) for each film_id
cursor.execute("""
    DELETE FROM film_insights
    WHERE insight_id NOT IN (
        SELECT MIN(insight_id)
        FROM film_insights
        GROUP BY film_id
    )
""")

conn.commit()
conn.close()


## Conver to JSON

In [117]:
import sqlite3
import json

def export_films_to_json(db_path="films.db", output_file="films.json"):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Optionally join with film_insights table to get short sentences
    # If each film has exactly one insight, you can do a LEFT JOIN:
    query = """
    SELECT f.id, 
           f.title, 
           f.release_year, 
           f.director, 
           f.box_office, 
           f.country,
           i.insight
      FROM films f
      LEFT JOIN film_insights i ON f.id = i.film_id
    ORDER BY f.id
    """
    
    cursor.execute(query)
    rows = cursor.fetchall()
    
    # Build a list of dictionaries
    # If a film has multiple insights, you'd see duplicates or handle differently
    films_data = []
    for row in rows:
        film_id = row[0]
        title = row[1] or ""
        release_year = row[2]
        director = row[3] or ""
        box_office = row[4]
        country = row[5] or ""
        insight = row[6] or ""
        
        # Convert box_office to a float if it's stored as text, or keep as float if already numeric
        try:
            box_office_val = float(box_office)
        except:
            # Fall back to None or 0 if parsing fails
            box_office_val = 0
        
        film_dict = {
            "id": film_id,
            "title": title,
            "release_year": release_year,
            "director": director,
            "box_office": box_office_val,
            "country": country,
            "insight": insight
        }
        
        films_data.append(film_dict)
    
    # Close DB connection
    conn.close()
    
    # Write to JSON file
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(films_data, f, indent=2)
    
    print(f"Exported {len(films_data)} records to {output_file}")

# Usage:
if __name__ == "__main__":
    export_films_to_json(db_path="films.db", output_file="films.json")


Exported 56 records to films.json
